<a href="https://colab.research.google.com/github/EmilSeyfullayev/Home-prices-analytics/blob/main/Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [138]:
!pip install gsheetsdb

In [139]:
from gsheetsdb import connect

In [140]:
conn = connect()

In [141]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1J1Lw1vGLEIee31gMLiE8Oihy9LuWSKhvuDdf9qSEKMo/edit?usp=sharing'

In [142]:
df = pd.read_sql(f'SELECT * FROM "{sheet_url}"', conn)

In [143]:
df.shape

(16218, 26)

In [144]:
len(df.drop_duplicates(subset='ids')) # No duplicate ids

16218

# Analytics

## Preprocessing

In [145]:
checkpoint_1 = df.iloc[:, :]

In [146]:
df = checkpoint_1.iloc[:, :]

In [147]:
df.columns.values

array(['ids', 'vipped_featured', 'prices', 'texts', 'categories',
       'floors', 'areas_m2', 'rooms', 'documents', 'credits', 'address',
       'district', 'latitudes', 'longitudes', 'ownerships', 'names',
       'emails', 'phone_numbers_appened', 'ad_number', 'watches',
       'ad_refreshed_date', 'agency_titles', 'photo_counts',
       'phot_pseudo_links', 'date_of_parsing', 'deal_ended'], dtype=object)

In [148]:
# Necessary columns
necessary_columns = [
                     "ids",
                     "vipped_featured", 
                     "prices",
                     "categories",
                     "floors",
                     "areas_m2",
                     "rooms",
                     "documents",
                     "credits",
                     "address",
                     "district",
                     "latitudes",
                     "longitudes",
                     "ownerships",
                     "watches",
                     "ad_refreshed_date",
                     "agency_titles",
                     "date_of_parsing",
                     "deal_ended"
]
df = df[necessary_columns]

In [149]:
df.head()

,ids,vipped_featured,prices,categories,floors,areas_m2,rooms,documents,credits,address,district,latitudes,longitudes,ownerships,watches,ad_refreshed_date,agency_titles,date_of_parsing,deal_ended
0,2348807,featuredvipped,54 000,Köhnə tikili,2 / 5,50 m²,2,var,var,"Ünvan: Bakı şəhəri, R.Şahsuvarov küç 31",Suraxanı r.; Hövsan q.,40.35582129,50.06357475,mülkiyyətçi,3363,01 Yanvar 2022,0,2022-01-03 11:36:55,0
1,1710054,vipped,289 000,Yeni tikili,5 / 17,180 m²,4,var,var,"Ünvan: Bakı şəhəri, Affiyədin Cəlilov küç.",Şah İsmayıl Xətai m.; Xətai r.,40.37891689,49.875682,mülkiyyətçi,7348,22 Dekabr 2021,0,2022-01-03 11:36:55,0
2,2524899,vipped,159 000,Yeni tikili,12 / 19,58 m²,2,var,var,"Ünvan: Bakı şəhəri, Azadlıq prospekti",ASAN Xidmət №1 ; TQDK ; Neftçi bazası ; Nəsimi r.,40.39841488,49.83920251,vasitəçi (agent),197,28 Dekabr 2021,0,2022-01-03 11:36:55,0
3,2529070,vipped,175 000,Yeni tikili,7 / 7,117 m²,3,var,var,"Ünvan: Bakı şəhəri, Xəqani Rüstəmov küçəsi 6",Əhmədli m.; Xətai r.,40.3816452,49.9584505,mülkiyyətçi,376,25 Dekabr 2021,0,2022-01-03 11:36:55,0
4,2541267,0,104 000,Yeni tikili,11 / 12,65 m²,2,var,xeyr,"Ünvan: Bakı şəhəri, S.S.Axundov küç.",Azadlıq Prospekti m.; Binəqədi r.; 8-ci mikror...,40.42705637,49.84432445,vasitəçi (agent),28,Bugün 15:20,New House 20 Yanvar daşınmaz əmlak agentliyi,2022-01-03 11:36:55,0


In [150]:
df.shape

(16218, 19)

### Vipped Featured ordinary ads

In [151]:
df['vipped_featured'].value_counts()

0                 15872
vipped              249
featuredvipped       77
featured             20
Name: vipped_featured, dtype: int64

In [152]:
ad_type = pd.get_dummies(df['vipped_featured'], prefix='ad_type', drop_first=True)
ad_type # ad_type_0 is reference category

,ad_type_featured,ad_type_featuredvipped,ad_type_vipped
0,0,1,0
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,0
...,...,...,...
16213,0,0,0
16214,0,0,0
16215,0,0,0
16216,0,0,0


In [153]:
df = pd.concat([df, ad_type], axis=1)
df.columns.values

array(['ids', 'vipped_featured', 'prices', 'categories', 'floors',
       'areas_m2', 'rooms', 'documents', 'credits', 'address', 'district',
       'latitudes', 'longitudes', 'ownerships', 'watches',
       'ad_refreshed_date', 'agency_titles', 'date_of_parsing',
       'deal_ended', 'ad_type_featured', 'ad_type_featuredvipped',
       'ad_type_vipped'], dtype=object)

### Price

In [154]:
df['prices'] = pd.to_numeric(
    df['prices'].apply(lambda x: str(x).replace(" ", ""))
)

### Categories

In [155]:
df['categories'].value_counts()

Yeni tikili     10900
Köhnə tikili     5312
0                   6
Name: categories, dtype: int64

In [156]:
categories = pd.get_dummies(df['categories'], drop_first=True) # 0 is reference category
categories

,Köhnə tikili,Yeni tikili
0,1,0
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
16213,0,1
16214,0,1
16215,0,1
16216,0,1


In [157]:
df = pd.concat([df, categories], axis=1)
df.columns.values

array(['ids', 'vipped_featured', 'prices', 'categories', 'floors',
       'areas_m2', 'rooms', 'documents', 'credits', 'address', 'district',
       'latitudes', 'longitudes', 'ownerships', 'watches',
       'ad_refreshed_date', 'agency_titles', 'date_of_parsing',
       'deal_ended', 'ad_type_featured', 'ad_type_featuredvipped',
       'ad_type_vipped', 'Köhnə tikili', 'Yeni tikili'], dtype=object)

### Floors

In [158]:
df['floors'].value_counts()

5 / 5      592
4 / 5      589
3 / 5      484
2 / 5      458
1 / 5      312
          ... 
23 / 26      1
22 / 27      1
7 / 26       1
9 / 24       1
15 / 28      1
Name: floors, Length: 341, dtype: int64

In [159]:
apartment_floor = df['floors'].apply(lambda x: str(x).split(" / ")[0])

buildings_floor = []

for i in df['floors']:
  try:
    buildings_floor.append(str(i).split(" / ")[1])
  except:
    buildings_floor.append(np.nan)

In [160]:
df['apartment_floor'] = apartment_floor
df['buildings_floor'] = buildings_floor

### Area m2

In [161]:
df['areas_m2'] = df['areas_m2'].apply(lambda x: float(x.rstrip(" m²")))

### Rooms

In [162]:
df['rooms'] = pd.to_numeric(df['rooms'])

### Documents and Credits availability

In [163]:
df['documents'].value_counts()

var    16212
0          6
Name: documents, dtype: int64

In [164]:
df[df['documents'] == "0"] # so I think to drop all rows with this condition

,ids,vipped_featured,prices,categories,floors,areas_m2,rooms,documents,credits,address,district,latitudes,longitudes,ownerships,watches,ad_refreshed_date,agency_titles,date_of_parsing,deal_ended,ad_type_featured,ad_type_featuredvipped,ad_type_vipped,Köhnə tikili,Yeni tikili,apartment_floor,buildings_floor
574,2540391,0,0,0,0,0.0,0,0,xeyr,0,None,0,0,0,0,0,0,2022-01-03 13:48:28,0,0,0,0,0,0,0,NaN
1809,2537858,0,0,0,0,0.0,0,0,xeyr,0,None,0,0,0,0,0,0,2022-01-03 16:45:50,0,0,0,0,0,0,0,NaN
2176,2517926,0,0,0,0,0.0,0,0,xeyr,0,None,0,0,0,0,0,0,2022-01-04 02:11:18,0,0,0,0,0,0,0,NaN
4480,2529047,0,0,0,0,0.0,0,0,xeyr,0,None,0,0,0,0,0,0,2022-01-04 08:33:26,0,0,0,0,0,0,0,NaN
11144,2518566,0,0,0,0,0.0,0,0,xeyr,0,None,0,0,0,0,0,0,2022-01-04 10:59:18,0,0,0,0,0,0,0,NaN
12459,2502400,0,0,0,0,0.0,0,0,xeyr,0,None,0,0,0,0,0,0,2022-01-04 14:07:11,0,0,0,0,0,0,0,NaN


In [167]:
df = df[~(df['documents'] == "0")].reset_index(drop=True)

In [168]:
df['documents'].value_counts() # drop whole column

var    16212
Name: documents, dtype: int64

In [169]:
df = df.drop('documents', axis=1)

In [170]:
df['credits'].value_counts()

xeyr    13341
var      2871
Name: credits, dtype: int64

In [175]:
credits = pd.get_dummies(df['credits'], prefix='Credit')
credits = credits['Credit_var'] # Credit_xeyr will be reference category

In [176]:
df = pd.concat([df, credits], axis=1)

### Address and District

In [180]:
df['address'].isna().sum()

0

In [185]:
len(df['address'].unique())

6495

In [193]:
df['address'].value_counts().head(10)

Ünvan: Bakı şəhəri, Gəncə prospekti       241
Ünvan: Bakı şəhəri, Mir Cəlal küç.        169
Ünvan: Bakı şəhəri, Məhəmməd Hadi küç.    151
Ünvan: Bakı şəhəri, Əsəd Əhmədov küç.     147
Ünvan: Bakı şəhəri, Azadlıq pr.           144
Ünvan: Bakı şəhəri, Gəncə pr.             135
Ünvan: Bakı şəhəri, Təbriz küç.           117
Ünvan: Bakı şəhəri, Şərifzadə küç.        112
Ünvan: Bakı şəhəri, Azadlıq prospekti     111
Ünvan: Bakı şəhəri, Gəncə prospekti        93
Name: address, dtype: int64

In [194]:
df['address'] = df['address'].apply(lambda x: x.lstrip("Ünvan: Bakı şəhəri, "))

In [197]:
df['address'].value_counts().head(10)

Gəncə prospekti       241
Mir Cəlal küç.        169
Məhəmməd Hadi küç.    151
Əsəd Əhmədov küç.     147
Azadlıq pr.           145
Gəncə pr.             135
Təbriz küç.           117
Azadlıq prospekti     116
Şərifzadə küç.        112
Gəncə prospekti        93
Name: address, dtype: int64

In [200]:
df['district'].value_counts().head(10)

Yasamal r.; Yeni Yasamal q.                                     811
Şah İsmayıl Xətai m.; Xətai r.                                  789
Xətai r.; Əhmədli q.                                            502
Nərimanov r.                                                    450
İnşaatçılar m.; Yasamal r.                                      371
Yasamal r.                                                      325
Nəriman Nərimanov m.; Nərimanov r.                              266
Nəriman Nərimanov m.; Nəriman Nərimanov parkı ; Nərimanov r.    242
Sabunçu r.; Bakıxanov q.                                        241
Azadlıq Prospekti m.; Binəqədi r.; 8-ci mikrorayon q.           230
Name: district, dtype: int64

In [206]:
def number_of_max_district_per_one_home():
  n = 0
  for i in df['district'].apply(lambda x: x.split("; ")):
    if len(i) > n:
      n = len(i)
  
  return n

number_of_max_district_per_one_home()
    


12

In [208]:
df['district'].apply(lambda x: x.split("; "))

0                             [Suraxanı r., Hövsan q.]
1                     [Şah İsmayıl Xətai m., Xətai r.]
2    [ASAN Xidmət №1 , TQDK , Neftçi bazası , Nəsim...
3                               [Əhmədli m., Xətai r.]
4    [Azadlıq Prospekti m., Binəqədi r., 8-ci mikro...
5    [Nəsimi bazarı , Koala parkı , Dostluq kinotea...
6    [Nəriman Nərimanov m., Nəriman Nərimanov parkı...
7                          [Suraxanı r., Qaraçuxur q.]
8                             [8 Noyabr m., Nəsimi r.]
9    [Axundov bağı , İqsadiyyat Universiteti , Dövl...
Name: district, dtype: object

In [223]:
districts_in_lists = df['district'].apply(lambda x: x.split("; "))

In [224]:
districts = []

for i in districts_in_lists:
  for z in i:
    districts.append(z.strip())

In [225]:
districts[:10]

['Suraxanı r.',
 'Hövsan q.',
 'Şah İsmayıl Xətai m.',
 'Xətai r.',
 'ASAN Xidmət №1',
 'TQDK',
 'Neftçi bazası',
 'Nəsimi r.',
 'Əhmədli m.',
 'Xətai r.']

In [226]:
len(districts)

44878

In [229]:
pd.Series(districts).value_counts().head(50)

Yasamal r.                        3884
Xətai r.                          3124
Nəsimi r.                         2974
Nərimanov r.                      2377
Binəqədi r.                       1412
Şah İsmayıl Xətai m.              1024
Nizami r.                         1005
Əhmədli q.                         960
İnşaatçılar m.                     921
Yeni Yasamal q.                    917
Yasamal q.                         787
Elmlər Akademiyası m.              743
Nəriman Nərimanov m.               730
Memar Əcəmi m.                     720
Səbail r.                          717
Həzi Aslanov m.                    621
28 May m.                          528
Nizami m.                          499
Yasamal bazarı                     486
8-ci kilometr q.                   456
ASAN Xidmət №1                     453
Nəriman Nərimanov parkı            422
8-ci mikrorayon q.                 418
Tibb Universiteti                  413
9-cu mikrorayon q.                 400
Gənclik m.               

In [217]:
len(set(districts))

181

In [230]:
districts_set = list(set(districts))

In [222]:
districts[:10]

['Yeni Yasamal q.',
 'Koroğlu m.',
 'Rəssamlıq Akademiyası',
 'Sabunçu r.',
 'Biləcəri q.',
 'Mehdiabad q.',
 'Suraxanı q.',
 'Qaraçuxur q.',
 'Hüseyn Cavid parkı',
 'Dərnəgül m.']